# Description

This notebook is used to request computation of Water Productivity for customized season period, crop multiplier and area from WaPOR's Gross Biomass Water Productivity (GBWP) or Net Biomass Water Productivity (NBWP) layers using WaPOR API.

You will need WaPOR API Token to use this notebook (See README.md)

# Import requests and json libraries

In [1]:
import requests
import json

# Login: Using APIToken to generate Accesstoken

In [2]:
APIToken=input('Enter your APIToken: ') 

request_url='https://io.apps.fao.org/gismgr/api/v1/iam/sign-in/'

request_headers = {
            'X-GISMGR-API-KEY': APIToken}

resp = requests.post(
                request_url,
                headers=request_headers)
resp_json = resp.json()
if resp_json['message']=='OK':
  AccessToken=resp_json['response']['accessToken']
  print('Access expires in 3600s')
else:
  print('Fail to get accessToken')


Enter your APIToken: Enter Your API Token when running this cell
Fail to get accessToken


# Send query CustomWaterProductivity

**Arguments** 'waterProductivity' type, 'startSeason', 'endSeason', 'resolution','shape'

In [3]:
url=r'https://io.apps.fao.org/gismgr/api/v1/query/'
request_json ={
  "type": "CustomWaterProductivity",
  "params": {
    "waterProductivity": "GBWP", #"GBWP" or "NBWP"
    "resolution": "100m", #"250m" or "100m" , maybe "30m" works for some area
    "startSeason": "2015-01", # "YYYY-DK" (Dekad)
    "endSeason": "2015-18", # "YYYY-DK" (Dekad)
    "shape": {
      "type": "Polygon", #define coordinates of the area in geojson format
      "coordinates": [
        [
          [
            37.20642480347329,
            9.879461495912246
          ],
          [
            36.49808605470977,
            7.56804031573655
          ],
          [
            37.84020157868276,
            5.219338148783827
          ],
          [
            40.0770607853044,
            5.293900122337882
          ],
          [
            41.97839111093279,
            7.232511434743303
          ],
          [
            41.68014321671657,
            8.313660051277097
          ],
          [
            39.89065585141926,
            7.605321302513577
          ],
          [
            38.5858213142233,
            7.344354395074386
          ],
          [
            38.51125934066925,
            8.649188932270341
          ],
          [
            37.20642480347329,
            9.879461495912246
          ]
        ]
      ]
    }
  }
}

request_headers = {
            'Authorization': "Bearer " + AccessToken}


response = requests.post(
                url,
                json=request_json,
                headers=request_headers)
response_json=response.json()
response_json

{'requestId': '37d64c9d-4eb5-436b-a7b3-bebee50d138a',
 'timestamp': 1576157978661,
 'runtime': 7,
 'status': 200,
 'message': 'OK',
 'response': {'workspaceCode': 'WAPOR_2',
  'code': '37d64c9d-4eb5-436b-a7b3-bebee50d138a',
  'type': 'WATER PRODUCTIVITY',
  'priority': 20,
  'additionalInfo': {'waterProductivity': 'GBWP',
   'resolution': '100m',
   'startSeason': '2015-01',
   'endSeason': '2015-18',
   'shape': {'type': 'Polygon',
    'coordinates': [[[37.20642480347329, 9.879461495912246],
      [36.49808605470977, 7.56804031573655],
      [37.84020157868276, 5.219338148783827],
      [40.0770607853044, 5.293900122337882],
      [41.97839111093279, 7.232511434743303],
      [41.68014321671657, 8.313660051277097],
      [39.89065585141926, 7.605321302513577],
      [38.5858213142233, 7.344354395074386],
      [38.51125934066925, 8.649188932270341],
      [37.20642480347329, 9.879461495912246]]]}},
  'userId': 'R6NHnGyt4TOzu3mvOkmxM8RLE2u1',
  'created': '2019-12-12 13:39:38',
  'stat

# Get job results

In [4]:
if response_json['message']=='OK':
  job_url = response_json['response']['links'][0]['href']
else:
  print('Fail to get job url')
response = requests.get(
                    job_url)
response.json()

{'requestId': '2c7b0236-2523-4a97-b10a-d5c3691dcd66',
 'timestamp': 1576157986396,
 'runtime': 2,
 'status': 200,
 'message': 'OK',
 'response': {'workspaceCode': 'WAPOR_2',
  'code': '37d64c9d-4eb5-436b-a7b3-bebee50d138a',
  'type': 'WATER PRODUCTIVITY',
  'priority': 20,
  'additionalInfo': {'waterProductivity': 'GBWP',
   'resolution': '100m',
   'startSeason': '2015-01',
   'endSeason': '2015-18',
   'shape': {'type': 'Polygon',
    'coordinates': [[[37.20642480347329, 9.879461495912246],
      [36.49808605470977, 7.56804031573655],
      [37.84020157868276, 5.219338148783827],
      [40.0770607853044, 5.293900122337882],
      [41.97839111093279, 7.232511434743303],
      [41.68014321671657, 8.313660051277097],
      [39.89065585141926, 7.605321302513577],
      [38.5858213142233, 7.344354395074386],
      [38.51125934066925, 8.649188932270341],
      [37.20642480347329, 9.879461495912246]]]}},
  'userId': 'R6NHnGyt4TOzu3mvOkmxM8RLE2u1',
  'created': '2019-12-12 13:39:38',
  'stat

# It will take a while for server to generate results. 

Using loop to check when processing is completed

In [5]:
while True:
  response = requests.get(
                    job_url)
  response_json=response.json()
  if response_json['response']['status']=='COMPLETED':
    GBWP=response_json['response']['output']['bwpDownloadUrl']
    TBP=response_json['response']['output']['tbpDownloadUrl']
    AETI=response_json['response']['output']['wtrDownloadUrl']
    break
print('Link to download GBWP',GBWP)
print('Link to download TBP',TBP)
print('Link to download AETI',AETI)

Link to download GBWP https://io.apps.fao.org/gismgr/download/79eda983-e710-4aa2-90bc-7c575b1a91e0/L2_GBWP_1501-1518.tif
Link to download TBP https://io.apps.fao.org/gismgr/download/79eda983-e710-4aa2-90bc-7c575b1a91e0/L2_TBP_1501-1518.tif
Link to download AETI https://io.apps.fao.org/gismgr/download/79eda983-e710-4aa2-90bc-7c575b1a91e0/L2_AETI_1501-1518.tif
